# Dogs Vs. Cats: Training a Covnet Using a Pretrained Model 

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np 
import matplotlib.pyplot as plt 

In [ ]:
# Import the VGG16 Model
conv_base = keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(180, 180, 3))